In [8]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
from dataclasses import dataclass

# Dataclasses for SDK5 Configuration inputs parameteres.
@dataclass
class ChirpComnCfg:
    digOutputSampRate: float # Sampling Rate decimator (sampling rate given by 100MHz/digoutputsamprate i.e digoutsamplerate = 8, sampling rate = 12.5MHz)
    digOutputBitsSel: int # Digital output sample bits select:
    # n important pelo momento
    """ 
        - 0 - Digital sample output is 12 MSB bits of DFE after rounding 4 LSBs. 
        - 1 - Digital sample output is 12 bits after rounding 3 LSBs & clipping 1 MSB. 
        - 2 - Digital sample output is 12 bits after rounding 2 LSBs & clipping 2 MSB. 
        - 3 - Digital sample output is 12 bits after rounding 1 LSBs & clipping 3 MSB. 
        - 4 - Digital sample output is 12 LSB bits after clipping 4 MSB. 
        - 5 - Digital sample output is 16 bits.
    """
    dfeFirSel: int # The final stage FIR filter's characteristics:
    # n important pelo momento
    """
        0 - Long Filter (90% visibility)
        1 - Shorter filter (80% visibility)
    """
    numOfAdcSamples: int # Number of ADC samples collected during ADC sampling time (needs to be power of 2)
    
    chirpTxMimoPatSel: int # ? modelos de estimacao de angulo diferentes (n importante no momento)
    """
        1 - enable TDM
        4 - enable BPM
    """
    
    chirpRampEndTime: float # ramp end time (us) 
    # bem importante !!!!!!!!!!!!!!!!!!!!!!!!
    """
        Ramp end time (us): This parameter is the sum of the ADC start time, ADC sampling time, and some ramp excess time.
        In other words, its the ramp period.
        Documentation says a minimum of:
        (chirpAdcSkipSamples + numOfAdcSamples + 7), for short filter (dfeFirSel = 1)
        (chirpAdcSkipSamples + numOfAdcSamples + 10), for long filter (dfeFirSel = 0)

    """
    
    chirpRxHpfSel: int # Chirp Profile HPF corner frequency:
    # High pass filter to filter out the bumper or anything too close to the radar.
    """
        - 0 - 175kHz HPF corner frequency 
        - 1 - 350kHz HPF corner frequency
        - 2 - 700kHz HPF corner frequency 
        - 3 - 1400kHz HPF corner frequency
    """

@dataclass
class ChirpTimingCfg:
    chirpIdleTime: float # Idle Time (us) in the chirp cicle before the frequency ramp
    chirpAdcSkipSamples: int # Number of samples skipped during ADC sampling. The ADC start time in us is computed based on the sampling rate. (0 - 63)
    chirpTxStartTime: float # Tx Start Time (us). Recommended Range: +- 5us
    chirpRfFreqSlope: float # Frequency Slope (MHz/us). (-399MHz/us to +399MHz/us)
    chirpRfFreqStart: float # Starting Frequency (GHz)

@dataclass
class ChannelCfg:
    rxChannelEn: float # Rx Antenna Mask; for 3 antennas, it is 0111b = 7
    txChannelEn: float # Tx Antenna; for 2 antenas it is 0011b = 3
    miscCtrl: float # ? SoC cascading (not supported with SDK)

@dataclass
class FrameCfg:
    numOfChirppsInBurst: int # Number of chirps in a burst (1 - 65535)
    numOfChirpsAccum: int # Number of chirps accumulated (0 - 64)
    burstPeriodicity: float # Burst periodicity (us)
    numOfBurstsInFrame: int # Number of bursts within a frame (1 - 4096)
    framePeriodicity: float # Frame Periodicity (ms)
    numOfFrames: int # Number of total frames (0 - 65535) (0 = infinite)

@dataclass
class FactoryCalibCfg:
    saveEnable: bool # Save calibration data to flash (0-1)
    restoreEnable: bool # Restore and load calibration data from FLASH
    rxGain: float # RX Gain in dB (30-40 strongly recommended)
    txBackoff: float # TX Backoff in dB (0 - 26)
    flashOffset: float # Address ofset in the flash to be used while saving or restoring calibration data

# Dataclasses for device specific configuration parameters
@dataclass
class ConfigurationParameters:
    StartFrequency: float # (GHz)
    FrequencySlope: float # (MHz/us)
    SamplingRate: float # (MHz)
    NumberofSamples: int # Number of samples
    NumberofChirpsLoops: int # Number of chirps loops
    IdleTime: float # (us)
    ADCValidStartTime: int # (us)
    RampEndTime: float # (us)
    FramePeriodicity: float # (ms)

# Dataclasses for all caracteristics calculated based in the parametres
@dataclass
class ConfigCaracteristics:
    MaximumRange: float # (m)
    MaximumVelocity: float # (m/s)
    ChirpTime: float # (us)
    RFDutyCycle: float # (%)
    ActiveChirpingTime: float # (ms)
    MaxBeatFrequency: float # IFmax (MHz)
    CarrierFrequency: float # fc (GHz)
    RangeResolution: float # (m)
    VelocityResolution: float # (m/s)
    ChirpRepetitionPeriod: float # Tr (us)
    ComplianceChripTime: float # (ms)
    RadarCubeSize: float # (kB)
    ValidSweepBand: float # B (MHz)
    EndFrequency: float # (GHz)

class MmWaveConfig:
    def __init__(self, 
                 chirp_comn_cfg: ChirpComnCfg,
                 chirp_timing_cfg: ChirpTimingCfg,
                 channel_cfg: ChannelCfg,
                 frame_cfg: FrameCfg,
                 factory_calib_cfg: FactoryCalibCfg):
        
        self.chirp_comn_cfg = chirp_comn_cfg
        self.chirp_timing_cfg = chirp_timing_cfg
        self.channel_cfg = channel_cfg
        self.frame_cfg = frame_cfg
        self.factory_calib_cfg = factory_calib_cfg
        
        self.nRxAntenas = bin(int(channel_cfg.rxChannelEn)).count("1")
        self.nTxAntenas = bin(int(channel_cfg.txChannelEn)).count("1")
        self.nVirtualAntenas = self.nRxAntenas * self.nTxAntenas
        
        # self.config_caracteristics = self.calculate_caracteristics()

    def save_to_file(self, file_path: str = None):
        """Saves the configuration to a file in the required format."""
        if file_path is None:
            file_path = "./mmwave_config.cfg"
        
        with open(file_path, 'w') as file:
            file.write(f"% This cfg file needs to be completed with SDK commands\n")
            file.write(f"% The commands can be found at <SDK_BASE_DIR/docs/mmwave_sdk_user_guide.pdf\n")
            
            # Write the configurations in the required format
            file.write(f"chirpComnCfg {self.chirp_comn_cfg.digOutputSampRate} {self.chirp_comn_cfg.digOutputBitsSel} "
                    f"{self.chirp_comn_cfg.dfeFirSel} {self.chirp_comn_cfg.numOfAdcSamples} "
                    f"{self.chirp_comn_cfg.chirpTxMimoPatSel} {self.chirp_comn_cfg.chirpRampEndTime} "
                    f"{self.chirp_comn_cfg.chirpRxHpfSel}\n")
            
            file.write(f"chirpTimingCfg {self.chirp_timing_cfg.chirpIdleTime} {self.chirp_timing_cfg.chirpAdcSkipSamples} "
                    f"{self.chirp_timing_cfg.chirpTxStartTime} {self.chirp_timing_cfg.chirpRfFreqSlope} "
                    f"{self.chirp_timing_cfg.chirpRfFreqStart}\n")
            
            file.write(f"channelCfg {self.channel_cfg.rxChannelEn} {self.channel_cfg.txChannelEn} "
                    f"{self.channel_cfg.miscCtrl}\n")
            
            file.write(f"frameCfg {self.frame_cfg.numOfChirppsInBurst} {self.frame_cfg.numOfChirpsAccum} "
                    f"{self.frame_cfg.burstPeriodicity} {self.frame_cfg.numOfBurstsInFrame} "
                    f"{self.frame_cfg.framePeriodicity} {self.frame_cfg.numOfFrames}\n")
            
            file.write(f"factoryCalibCfg {self.factory_calib_cfg.saveEnable} {self.factory_calib_cfg.restoreEnable} "
                    f"{self.factory_calib_cfg.rxGain} {self.factory_calib_cfg.txBackoff} "
                    f"{self.factory_calib_cfg.flashOffset}\n")

    # def calculate_caracteristics(self):
    #     """Calculates the caracteristics of the radar based on the configuration."""
        
    #     c = 3e8 # Speed of light (m/s)
        
    #     MaximumRange = 
    #     MaximumVelocity = 
    #     ChirpTime = 
    #     RFDutyCycle = 
    #     ActiveChirpingTime = 
    #     MaxBeatFrequency = 
    #     CarrierFrequency = 
    #     RangeResolution = 
    #     VelocityResolution = 
    #     ChirpRepetitionPeriod = 
    #     ComplianceChripTime = 
    #     RadarCubeSize = 
    #     ValidSweepBand = 
    #     EndFrequency = 
        
    #     return ConfigCaracteristics(
    #         MaximumRange=MaximumRange,
    #         MaximumVelocity=MaximumVelocity,
    #         ChirpTime=ChirpTime,
    #         RFDutyCycle=RFDutyCycle,
    #         ActiveChirpingTime=ActiveChirpingTime,
    #         MaxBeatFrequency=MaxBeatFrequency,
    #         CarrierFrequency=CarrierFrequency,
    #         RangeResolution=RangeResolution,
    #         VelocityResolution=VelocityResolution,
    #         ChirpRepetitionPeriod=ChirpRepetitionPeriod,
    #         ComplianceChripTime=ComplianceChripTime,
    #         RadarCubeSize=RadarCubeSize,
    #         ValidSweepBand=ValidSweepBand,
    #         EndFrequency=EndFrequency
    #     )

    @classmethod
    def parse_mmwave_config(cls, file_content: str):
        """Parses the configuration from the provided file content."""
        lines = file_content.strip().split('\n')

        # Dictionaries to hold parsed values
        chirp_comn_cfg = chirp_timing_cfg = channel_cfg = frame_cfg = factory_calib_cfg = None

        # Parse each line and map values to the appropriate config object
        for line in lines:
            parts = line.split()
            if line.startswith("chirpComnCfg"):
                chirp_comn_cfg = ChirpComnCfg(*map(float, parts[1:]))
            elif line.startswith("chirpTimingCfg"):
                chirp_timing_cfg = ChirpTimingCfg(*map(float, parts[1:]))
            elif line.startswith("channelCfg"):
                channel_cfg = ChannelCfg(*map(float, parts[1:]))
            elif line.startswith("frameCfg"):
                frame_cfg = FrameCfg(*map(float, parts[1:]))
            elif line.startswith("factoryCalibCfg"):
                factory_calib_cfg = FactoryCalibCfg(*map(float, parts[1:]))

        # Create and return an instance of MmWaveConfig
        return cls(
            chirp_comn_cfg=chirp_comn_cfg,
            chirp_timing_cfg=chirp_timing_cfg,
            channel_cfg=channel_cfg,
            frame_cfg=frame_cfg,
            factory_calib_cfg=factory_calib_cfg
        )

    @classmethod
    def print_results(cls, config_caracteristics: ConfigCaracteristics):
        print(f"Maximum Range: {config_caracteristics.MaximumRange} m")
        print(f"Maximum Velocity: {config_caracteristics.MaximumVelocity} m/s")
        print(f"Chirp Time: {config_caracteristics.ChirpTime} us")
        print(f"RF Duty Cycle: {config_caracteristics.RFDutyCycle} %")
        print(f"Active Chirping Time: {config_caracteristics.ActiveChirpingTime} ms")
        print(f"Max Beat Frequency: {config_caracteristics.MaxBeatFrequency} MHz")
        print(f"Carrier Frequency: {config_caracteristics.CarrierFrequency} GHz")
        print(f"Range Resolution: {config_caracteristics.RangeResolution} m")
        print(f"Velocity Resolution: {config_caracteristics.VelocityResolution} m/s")
        print(f"Chirp Repetition Period: {config_caracteristics.ChirpRepetitionPeriod} us")
        print(f"Compliance Chirp Time: {config_caracteristics.ComplianceChripTime} ms")
        print(f"Radar Cube Size: {config_caracteristics.RadarCubeSize} kB")
        print(f"Valid Sweep Band: {config_caracteristics.ValidSweepBand} MHz")
        print(f"End Frequency: {config_caracteristics.EndFrequency} GHz")

def load_mmwave_config_from_file(file_path: str) -> MmWaveConfig:
    with open(file_path, 'r') as file:
        file_content = file.read()
    return MmWaveConfig.parse_mmwave_config(file_content)


In [10]:
# I'll first read the file content to understand its structure and then proceed to parse it.
file_path = './profile_2024_10_10T13_21_31_415.cfg'

# Parsing the configuration file
mmwave_config = load_mmwave_config_from_file(file_path)

mmwave_config.__dict__

{'chirp_comn_cfg': ChirpComnCfg(digOutputSampRate=33.0, digOutputBitsSel=0.0, dfeFirSel=0.0, numOfAdcSamples=128.0, chirpTxMimoPatSel=4.0, chirpRampEndTime=50.0, chirpRxHpfSel=0.0),
 'chirp_timing_cfg': ChirpTimingCfg(chirpIdleTime=6.0, chirpAdcSkipSamples=19.0, chirpTxStartTime=0.0, chirpRfFreqSlope=18.39, chirpRfFreqStart=76.0),
 'channel_cfg': ChannelCfg(rxChannelEn=7.0, txChannelEn=3.0, miscCtrl=0.0),
 'frame_cfg': FrameCfg(numOfChirppsInBurst=2.0, numOfChirpsAccum=0.0, burstPeriodicity=227.0, numOfBurstsInFrame=20.0, framePeriodicity=200.0, numOfFrames=0.0),
 'factory_calib_cfg': FactoryCalibCfg(saveEnable=1.0, restoreEnable=0.0, rxGain=30.0, txBackoff=0.0, flashOffset=0.0),
 'nRxAntenas': 3,
 'nTxAntenas': 2,
 'nVirtualAntenas': 6}

In [11]:
# mmwave_config.save_to_file('./oi.cfg')
# mmwave_config.print_results(mmwave_config.config_caracteristics)

In [22]:
slope = mmwave_config.chirp_timing_cfg.chirpRfFreqSlope # MHz/us
f0 = mmwave_config.chirp_timing_cfg.chirpRfFreqStart # GHz
nADC = mmwave_config.chirp_comn_cfg.numOfAdcSamples # number of samples taken with the ADC (always power of two)
fs = 100/mmwave_config.chirp_comn_cfg.digOutputSampRate # MHz

T_chirp_ramp = mmwave_config.chirp_comn_cfg.chirpRampEndTime # 

chirp_idle_time = mmwave_config.chirp_timing_cfg.chirpIdleTime # us
n_skiped_chirps = mmwave_config.chirp_timing_cfg.chirpAdcSkipSamples


n_chirps_in_burst = mmwave_config.frame_cfg.numOfChirppsInBurst
n_bursts_in_frame = mmwave_config.frame_cfg.numOfBurstsInFrame
T_frame = mmwave_config.frame_cfg.framePeriodicity # ms
